In [1]:
# The modules we're going to use
from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, convolutional, pooling, Flatten, Dropout, ZeroPadding2D
from keras.optimizers import *

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# When you execute a code to plot with a simple SHIFT-ENTER, the plot will be shown directly under the code cell
%matplotlib inline

Using TensorFlow backend.


In [2]:
# Get all the Data and Reshape with 48x48 pixel size
X_train = np.genfromtxt('data/train_data.csv', delimiter=',')
n, m = X_train.shape
X_train = X_train.reshape(n, 48, 48)

X_test = np.genfromtxt('data/test_data.csv' , delimiter=',')
n, m = X_test.shape
X_test = X_test.reshape(n, 48, 48)

Y_train = np.genfromtxt('data/train_target.csv', delimiter=',')

print("Before processing: X_train: ", X_train.shape)
print("Before processing: X_test: ", X_test.shape)
print("Before processing Y_train: ", Y_train.shape)

Before processing: X_train:  (16175, 48, 48)
Before processing: X_test:  (3965, 48, 48)
Before processing Y_train:  (16175,)


In [3]:
# Pre-processing as mentioned in Demo
X_train = X_train.reshape(-1,1, 48,48)
X_test = X_test.reshape(-1,1, 48,48)
Y_train = np_utils.to_categorical(Y_train,3)

print("After processing: X_train: ", X_train.shape)
print("After processing: X_test: ", X_test.shape)
print("After processing Y_train: ", Y_train.shape)

After processing: X_train:  (16175, 1, 48, 48)
After processing: X_test:  (3965, 1, 48, 48)
After processing Y_train:  (16175, 3)


In [4]:
# Tried this approch but accuracy was very low ~ 27.45%

# Pre-processing
# X_train = X_train.reshape(X_train.shape[0], 48, 48 , 1)
# X_test = X_test.reshape(X_test.shape[0], 48, 48 , 1)

# X_train = X_train.astype('float32')
# X_test = X_test.astype('float32')
# X_train /= 255
# X_test /= 255

# Y_train = np_utils.to_categorical(Y_train,3)

# # Add 1st Layer
# model.add(convolutional.Conv2D(filters=64, kernel_size=(3,3), strides=(1, 1), padding='same',    
#     input_shape=(1,48,48), activation='relu'))
# model.add(convolutional.Conv2D(64, (3,3), padding='same'))
# model.add(pooling.MaxPooling2D(pool_size=(2, 2), padding='same'))

# # Add 2nd Layer
# model.add(convolutional.Conv2D(128, (3,3), padding='same', activation='relu'))
# model.add(convolutional.Conv2D(128, (5,5), padding='same', activation='relu'))
# model.add(pooling.MaxPooling2D(pool_size=(2,2),padding='same'))

# # Add 3rd Layer
# model.add(convolutional.Conv2D(256, (3,3), padding='same', activation='relu'))
# model.add(convolutional.Conv2D(256, (5,5), padding='same', activation='relu'))
# model.add(pooling.MaxPooling2D(pool_size=(2,2),padding='same'))

# Result
# Validation Accuracy = 27.45% WITH 20 Epochs and 32 Batch Size 

# Possible Problem
# Data Pre-Processing can be more optimized

In [5]:
# Create a Neural Network
model = Sequential()

# Add 1st Layer
model.add(convolutional.Conv2D(filters=32, kernel_size=(3,3), strides=(1, 1), padding='same',    
    input_shape=(1,48,48), activation='relu'))
model.add(convolutional.Conv2D(32, (3,3), padding='same'))
model.add(pooling.MaxPooling2D(pool_size=(2, 2), padding='same'))

# Add 2nd Layer
model.add(convolutional.Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(convolutional.Conv2D(64, (5,5), padding='same', activation='relu'))
model.add(convolutional.Conv2D(64, (5,5), padding='same', activation='relu'))
model.add(pooling.MaxPooling2D(pool_size=(2,2),padding='same'))

# Add 3rd Layer
model.add(convolutional.Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(convolutional.Conv2D(128, (5,5), padding='same', activation='relu'))
model.add(convolutional.Conv2D(128, (5,5), padding='same', activation='relu'))
model.add(pooling.MaxPooling2D(pool_size=(2,2),padding='same'))

# Add flat Layer and Softmax
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(3, activation='softmax'))

# Compile Model using Assign Optimizer (Tried Different Type of Optimizers)
adam = Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=1e-6)
# sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# opt = rmsprop(lr=0.0001, decay=1e-6)

# Used Demo's Adam, in-Built 'adam' was not giving most optimal results
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])

# Print Model Summary
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 48, 32)         13856     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 1, 48, 32)         9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 1, 24, 32)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 1, 24, 64)         18496     
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 24, 64)         102464    
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 1, 24, 64)         102464    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 1, 12, 64)         0         
__________

In [6]:
# Training Model and perform 80-20 split check
print("Start Training...")
model.fit(X_train,Y_train, epochs=50, batch_size=128, verbose=1, validation_split=0.2)

print("Finished Training!")

Start Training...
Train on 12940 samples, validate on 3235 samples
Epoch 1/50
12940/12940 [==============================] - 26s - loss: 1.0648 - acc: 0.4468 - val_loss: 1.0417 - val_acc: 0.4510

KeyboardInterrupt: 

In [ ]:
# Testing
loss, accuracy = model.evaluate(X_train, Y_train, verbose=1)
print('Test Loss:', loss)
print('Test Accuracy', accuracy)

In [ ]:
# Make Predictions using X-Test
predictions = model.predict(X_test, verbose=1)
output = predictions.argmax(axis=1)

print(output)

In [ ]:
# Store data on file
print("Creating Submission File...")
f = open('data/hardeep_final_submission.csv','w')
f.write('Id,Category\n') 
for i in range(0, X_test.shape[0]):
    f.write(str(i) + ',' + str(output[i]) + '\n')
f.close()
print("Submission File Created Successfully!")